In [18]:
# %pip install wolframalpha

In [1]:
import os
import openai
import json

from prompts import SIMPLE_PHYSICS_NUMERICAL_MCQ_PROMPT, SIMPLE_TEXT_MCQ_PROMPT, CALCULATION_CHECK_PROMPT
from helper import get_results_from_wolfram
from utils import add_dicts

from dotenv import load_dotenv
load_dotenv()

True

In [20]:
# import requests

# def query_wolframalpha(query):
#     base_url = "https://www.wolframalpha.com/api/v1/llm-api"
#     appid = os.environ.get("WOLFRAM_APP_ID")

#     params = {
#         "input": query,
#         "appid": appid
#     }
    
#     response = requests.get(base_url, params=params)

#     print(response)
    
#     if response.status_code == 200:
#         return response.text
#     else:
#         return f"Error: {response.status_code}"

# # Example usage:
# query = "gamma = 1.4; P1 = 2; V1 = 4; V2 = 2; P2 = P1*V1^gamma/V2^gamma"
# # query = "4+5"
# query_result = query_wolframalpha(query)
# print(query_result)

In [21]:
# get_results_from_wolfram("calculate for P2, gamma = 1.4; P1 = 2; V1 = 4; V2 = 2; P2 = P1*V1^gamma/V2^gamma")

In [2]:
functions = [
    {
      "name": "get_results_from_wolfram",
      "description": "This function will return a list of accurate calculation results for the mathematical queries",
      "parameters": {
        "type": "object",
        "properties": {
          "queries": {
            "type": "string",
            "description": """the list of mathematical expressions for example - "['2^2 + 3', 'ln(5) + 2*4']"."""
          }
        }
      },
        "required": ["queries"]
    }
] 

In [3]:
# system_message = {"role": "system", "content": "You are a help assistant"}
system_message = {"role": "system", "content": SIMPLE_PHYSICS_NUMERICAL_MCQ_PROMPT}
# user_message = {"role": "user", "content": SIMPLE_NUMERICAL_MCQ_PROMPT}

conversation = [system_message]

def generate_numerical_mcqs(conversation, context, n=1, example = None, model_id = "gpt-4-0613", get_more_complex = 0, return_usage = False):

    global total_usage
    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}

    user_prompt = f"""
    CONTEXT_TEXT:
    //context//

    {context}

    //context//
   
    N: {n}

    EXAMPLE:
    {example if example else "NA"}

    QUESTIONS: 
    """

    user_message = {'role': 'user', "content": user_prompt}
    conversation.append(user_message)

    response = openai.ChatCompletion.create(
                                    model= model_id,
                                    messages = conversation,
                                    temperature = 1
                                    )
    
    total_usage = add_dicts(total_usage, dict(response.usage))

    # if get_more_complex > 0:
    #     user_new_message = {"role": "user", "content": f"Generate {get_more_complex} more questions in the same format but should be more complex"}
    #     conversation.append(user_new_message)

    #     response = openai.ChatCompletion.create(
    #                                 model= model_id,
    #                                 messages = conversation,
    #                                 functions = functions,
    #                                 temperature = 1
    #                                 )
        
    #     conversation.append(response.choices[0].message)
    #     total_usage = add_dicts(total_usage, dict(response.usage))

    output = response.choices[0].message.content

    if return_usage:
        return output, total_usage
    
    return output

context = """
Thermodynamics

Definitions:
1. System: A part of the universe under study.
2. Surroundings: Everything other than the system.
3. Thermodynamic State: The description of a system in terms of properties like pressure, volume, temperature, etc.
4. State Functions: Properties that depend only on the state of the system, not the path. Examples: U (Internal energy), H (Enthalpy), S (Entropy), and G (Gibbs free energy).
5. Path Functions: Depend on the path taken during a change, like heat (q) and work (w).
6. First Law of Thermodynamics: ΔU = q + w, where ΔU is the change in internal energy, q is heat, and w is work.
7. Isothermal Process: Process in which temperature remains constant.
8. Adiabatic Process: No heat exchange with surroundings. 

Formulas:
1. w = -P_ext x ΔV 
   - Work done during a quasi-static isothermal expansion/compression.
2. For an ideal gas undergoing isothermal expansion or compression:
   w = -nRT ln(Vf/Vi)
3. For an adiabatic process in an ideal gas:
   PV^γ = constant
   where γ is the ratio of specific heats.

"""

example = {
        "q": "An ideal gas undergoes an isothermal expansion. If the initial volume Vi is 2 liters and the final volume Vf is 5 liters, the number of moles n is 3, the gas constant R is 8.314 J/mol·K, and the temperature T is 300 K, calculate the work done during this process.",
        "c": ["-5480.26 J", "-6856.24 J", "-7200.00 J", "-6500.00 J"],
        "a": "-6856.24 J"
    }

output = generate_numerical_mcqs(
                        conversation,
                        context = context,
                        n = 3,
                        # example = example,
                        # get_more_complex=2,
                        return_usage=True
                        )

print(output)

('{\n    "mcqs": [\n        {\n        "q": "Given a system in an isothermal expansion, with P_ext = 1 atm, ΔV = 2 L, (consider 1 atm = 101.3 J/L). What is the work done by the system?", \n        "c": ["-202.6 J", "-101.3 J", "202.6 J", "101.3 J"],\n        "a": "-202.6 J"\n        },\n        {\n        "q": "If a particular ideal gas undergoes an isothermal expansion from volume 2L to 5L, and there is 2 moles of the gas at a temperature of 298 K, calculate the work done in processing the gas. (8.314 J/mol.K as R)", \n        "c": ["-3638.7 J", "3638.7 J", "-1819.3 J", "1819.3 J"],\n        "a": "-3638.7 J"\n        },\n        {\n        "q": "Which of the following statements best describes an adiabatic process in thermodynamics?",\n        "c": ["The system exchanges heat with the surroundings", "The system neither gains nor loses heat from the surroundings", "The system\'s temperature remains constant during the process", "All of the above"],\n        "a": "The system neither gai

In [4]:
print(output[0])

{
    "mcqs": [
        {
        "q": "Given a system in an isothermal expansion, with P_ext = 1 atm, ΔV = 2 L, (consider 1 atm = 101.3 J/L). What is the work done by the system?", 
        "c": ["-202.6 J", "-101.3 J", "202.6 J", "101.3 J"],
        "a": "-202.6 J"
        },
        {
        "q": "If a particular ideal gas undergoes an isothermal expansion from volume 2L to 5L, and there is 2 moles of the gas at a temperature of 298 K, calculate the work done in processing the gas. (8.314 J/mol.K as R)", 
        "c": ["-3638.7 J", "3638.7 J", "-1819.3 J", "1819.3 J"],
        "a": "-3638.7 J"
        },
        {
        "q": "Which of the following statements best describes an adiabatic process in thermodynamics?",
        "c": ["The system exchanges heat with the surroundings", "The system neither gains nor loses heat from the surroundings", "The system's temperature remains constant during the process", "All of the above"],
        "a": "The system neither gains nor loses heat 

In [5]:
# calculation check and improvements

def calculation_checking(input, model_id = "gpt-4-0613"):

    global total_usage
    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
    
    input_prompt = f"""

    INPUT: 

    {input}

    OUTPUT: 

    """ 
    
    system_message = {"role": "system", "content": CALCULATION_CHECK_PROMPT}
    conversation = [system_message]
    input_message = {'role': 'user', 'content': input_prompt}
    conversation.append(input_message)

    response = openai.ChatCompletion.create(
                                    model= model_id,
                                    messages = conversation,
                                    functions = functions,
                                    function_call = {'name': "get_results_from_wolfram"},
                                    temperature = 1
                                    )
    
    def response_with_function(response, conversation):

        global total_usage

        print("calling function".upper())
        function_call = response.choices[0]['message']['function_call']
        conversation.append(response.choices[0]['message'])

        if function_call["name"] == "get_results_from_wolfram":
            queries = json.loads(function_call["arguments"])["queries"]
            print("QUERIES", queries)

            if isinstance(queries, list):
                queries = queries
            else:
                queries = queries.split(",")

            try:
                print("good queries", queries)
                result = get_results_from_wolfram(queries)
            except:
                raise Exception("error from wolfram")

        else:
            raise "some error in function name"

        print("result from function".upper(), str(result))
        conversation.append({'role': 'function', "name": function_call["name"], "content": str(result)})

        response = openai.ChatCompletion.create(
            model= model_id,
            messages = conversation,
            functions = functions,
            temperature = 0.2
        )
        conversation.append(response.choices[0].message)
        total_usage = add_dicts(total_usage, dict(response.usage))
        return response, conversation
    
    response, conversation = response_with_function(response, conversation)
    
    output = response.choices[0].message.content

    return output



input = output[0]
    
out = calculation_checking(conversation, input)

print(out)

CALLING FUNCTION
QUERIES ['1 atm * 2 L', '2 moles * 8.314 J/mol.K * ln(5L/2L)']
good queries ['1 atm * 2 L', '2 moles * 8.314 J/mol.K * ln(5L/2L)']
RESULT FROM FUNCTION {'1 atm * 2 L': '2 atm L (atmosphere liters)', '2 moles * 8.314 J/mol.K * ln(5L/2L)': '15.236 J/K (joules per kelvin difference)'}

    {
    "mcqs": [
        {
        "q": "Given a system in an isothermal expansion, with P_ext = 1 atm, ΔV = 2 L, (consider 1 atm = 101.3 J/L). What is the work done by the system?", 
        "c": ["-202.6 J", "-101.3 J", "202.6 J", "101.3 J"],
        "a": "202.6 J"
        },
        {
        "q": "If a particular ideal gas undergoes an isothermal expansion from volume 2L to 5L, and there is 2 moles of the gas at a temperature of 298 K, calculate the work done in processing the gas. (8.314 J/mol.K as R)", 
        "c": ["-3638.7 J", "3638.7 J", "-1819.3 J", "1819.3 J"],
        "a": "3638.7 J"
        },
        {
        "q": "Which of the following statements best describes an adiab

In [26]:
# text = """In the words of Thomas DeQuincey, “It is notorious that the memory strengthens as 
# you lay burdens upon it.” If, like most people, you have trouble recalling the names of
# those you have just met, try this: The next time you are introduced, plan to remember 
# the names. Say to yourself, “I’ll listen carefully; I’ll repeat each person’s name to be sure
# I have it, and I will remember.” You’ll discover how effective this technique is and 
# probably recall those names for the rest of your life"""

context = """
Many people have owned, or have heard of, traditional “piggy banks,” coin banks 
shaped like pigs. A logical theory about how this tradition started might be that because 
pigs often symbolize greed, the object is to “fatten” one's piggy bank with as much
money as possible. However, while this idea makes sense, it is not the correct
origin of the term. The genesis of the piggy bank is the old English word “pygg”, which 
was a common kind of clay hundreds of years ago in England. People used pots and 
jars made out of this red “pygg” clay for many different purposes in their homes. 
Sometimes they kept their money in one of the pots, and this was known as a pygg 
bank. Over the years, because “pygg” and “pig” sounded the same, glaziers began
making novelty banks out of pottery in the shape of a pig as a kind of joke. These banks 
were given as gifts and exported to countries where people spoke other languages and 
where no one had ever heard of pygg clay. The tradition caught on all over the world, 
and today piggy banks come in all colors and are made of all kinds of materials, 
including plastic.
"""

In [27]:
# name = "get_results_from_wolfram"
# args = json.loads("\n{\n\"query\": \"3*8.314*300*ln(5/2)\"\n}")

# exec(f"{name}" + "(args["query"])})

## Thermodynamics

### Definitions:
1. **System**: A part of the universe under study.
2. **Surroundings**: Everything other than the system.
3. **Thermodynamic State**: The description of a system in terms of properties like pressure, volume, temperature, etc.
4. **State Functions**: Properties that depend only on the state of the system, not the path. Examples: U (Internal energy), H (Enthalpy), S (Entropy), and G (Gibbs free energy).
5. **Path Functions**: Depend on the path taken during a change, like heat (q) and work (w).
6. **First Law of Thermodynamics**: ΔU = q + w, where ΔU is the change in internal energy, q is heat, and w is work.
7. **Isothermal Process**: Process in which temperature remains constant.
8. **Adiabatic Process**: No heat exchange with surroundings. 

### Formulas:
1. \( w = -P_{ext} \times ΔV \) : Work done during a quasi-static isothermal expansion/compression.
2. For an ideal gas undergoing isothermal expansion or compression, \( w = -nRT \ln(\frac{V_f}{V_i}) \).
3. For an adiabatic process in an ideal gas, \( PV^γ = \) constant, where γ is the ratio of specific heats.
